In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="BrownUniversityLibrary")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df = pd.read_csv('RI_colleges.csv')
df['address'] = df[['location_address1',
                                     'location_city', 
                                     'location_state']].agg(','.join, axis=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('99 Main St,Franklin,MA',), **{}).
Traceback (most recent call last):
  File "/Users/eriq/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/eriq/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/eriq/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  Fi

In [2]:
missing_df = df[df['point'].isnull()]
missing_df['location'] = missing_df['name'].apply(geocode)
missing_df['point'] = missing_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_4538/227648415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df['location'] = missing_df['name'].apply(geocode)
/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_4538/227648415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df['point'] = missing_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,contact_fax,contact_email,group_ID,listing_order,sch_sub_type_ID,sch_sub_type_name,role_sort_order,source,address,point
12,3215,42.0,70504,NaN,Central Maine Community College,Central Maine Community Colleg,CMCC,2,School,Y,...,NaN,enroll@cmcc.edu,NaN,NaN,24,State Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"1250 Turner Street,Auburn,MN","(44.1321115, -70.23206239109422, 0.0)"
17,3206,42.0,70502,NaN,Dean College,Dean College,Dean College,2,School,Y,...,NaN,admissions@dean.edu,NaN,NaN,24,State Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"99 Main St,Franklin,MA","(42.084653149999994, -71.39983777231043, 0.0)"


In [7]:
missing_df.columns(['name', 'point'])

TypeError: 'Index' object is not callable

In [3]:
new_df = pd.concat([df, missing_df], axis=0)
new_df = new_df.drop_duplicates(subset='org_ID', keep='last')
new_df

,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,contact_fax,contact_email,group_ID,listing_order,sch_sub_type_ID,sch_sub_type_name,role_sort_order,source,address,point
0,3585,42.0,70506,NaN,Brigham Young University,Brigham Young University,Brigham Young U,2,School,Y,...,NaN,NaN,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"Brigham Young University,Provo,UT","(40.25501105, -111.64968264312222, 0.0)"
1,1102,NaN,28501,NaN,Brown University ...,Brown University,Brown Univ.,2,School,Y,...,(401) 863-7737,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Prospect Street ,Providence,RI","(41.8261477, -71.40384783120373, 0.0)"
2,1174,NaN,31501,NaN,Bryant University,Bryant University,Bryant Univ,2,School,Y,...,(401) 232-6709,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1150 Douglas Pike,Smithfield,RI","(41.9184668, -71.5365125, 0.0)"
3,3452,42.0,70505,NaN,Clarkson University,Clarkson University,Clarkson Univ,2,School,Y,...,NaN,NaN,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"8 Clarkson Ave,Potsdam,NY","(44.663631761589166, -74.99410567557055, 0.0)"
4,1105,NaN,28504,NaN,Johnson and Wales University,Johnson and Wales University,Johnson Wales,2,School,Y,...,(401) 598-1833,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"8 Abbott Park Place,Providence,RI","(41.825152599999996, -71.42491442533371, 0.0)"
5,704,NaN,21502,NaN,Naval War College,Naval War College,Naval War,2,School,Y,...,(401) 841-1297,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"686 Cushing Road,Newport,RI","(41.5076757, -71.3291036, 0.0)"
6,1376,NaN,35506,NaN,New England Institute of Technology ...,NE Institute of Technology,NEIT,2,School,Y,...,(401) 738-8990,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 New England Tech Boulevard,East Greenwich,RI","(41.6616, -71.50270757908407, 0.0)"
7,1103,NaN,28502,NaN,Providence College,Providence College,Providence Col.,2,School,Y,...,(401) 865-2105,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"549 River Avenue,Providence,RI","(41.843295833333336, -71.44055416666667, 0.0)"
8,1104,NaN,28503,NaN,Rhode Island School of Design,Rhode Island School of Design,RISD,2,School,Y,...,(401) 454-6406,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"2 College Street,Providence,RI","(41.826021, -71.407026, 0.0)"
9,1602,NaN,96501,NaN,Roger Williams University,Roger Williams University,Roger Williams,2,School,Y,...,(401) 254-3158,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Old Ferry Road,Bristol,RI","(41.642946, -71.259916, 0.0)"


In [43]:
df[df['point'].isnull()]['name']

13          Community College of R.I. - Flanagan Campus
14            Community College of R.I. - Knight Campus
15            Community College of R.I. - Liston Campus
16      Community College of R.I.-Newport County Campus
22    URI/Alan Shawn Feinstein College of Continuing...
Name: name, dtype: object

In [46]:
from geopy.geocoders import GeoNames
geolocator2 = GeoNames(username='rkique')
geocode2 = RateLimiter(geolocator2.geocode, min_delay_seconds=1)

In [49]:
missing_df = df[df['point'].isnull()]
missing_df['address'] = missing_df[['location_address1',
                                     'location_city', 
                                     'location_state']].agg(','.join, axis=1)
missing_df['location'] = missing_df['address'].apply(geocode)
missing_df['point'] = missing_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_25098/2524696638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df['address'] = missing_df[['location_address1',
/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_25098/2524696638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df['location'] = missing_df['address'].apply(geocode)
/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_25098/2524696638.py:6: SettingWithCopyWarning: 
A value is trying to be set on

In [51]:
missing_df['point']

13            (41.8979117, -71.45531186862678, 0.0)
14    (41.713034199999996, -71.48012695911763, 0.0)
15                     (41.80732, -71.4137023, 0.0)
16                     (41.515833, -71.312563, 0.0)
22                     (41.823274, -71.414212, 0.0)
Name: point, dtype: object

In [10]:
new_df = pd.concat([df, missing_df], axis=0)
new_df = new_df.drop_duplicates(subset='org_ID', keep='last')
new_df

,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,contact_fax,contact_email,group_ID,listing_order,sch_sub_type_ID,sch_sub_type_name,role_sort_order,source,address,point
0,3585,42.0,70506,NaN,Brigham Young University,Brigham Young University,Brigham Young U,2,School,Y,...,NaN,NaN,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"Brigham Young University,Provo,UT","(40.25501105, -111.64968264312222, 0.0)"
1,1102,NaN,28501,NaN,Brown University ...,Brown University,Brown Univ.,2,School,Y,...,(401) 863-7737,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Prospect Street ,Providence,RI","(41.8261477, -71.40384783120373, 0.0)"
2,1174,NaN,31501,NaN,Bryant University,Bryant University,Bryant Univ,2,School,Y,...,(401) 232-6709,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1150 Douglas Pike,Smithfield,RI","(41.9184668, -71.5365125, 0.0)"
3,3452,42.0,70505,NaN,Clarkson University,Clarkson University,Clarkson Univ,2,School,Y,...,NaN,NaN,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"8 Clarkson Ave,Potsdam,NY","(44.663631761589166, -74.99410567557055, 0.0)"
4,1105,NaN,28504,NaN,Johnson and Wales University,Johnson and Wales University,Johnson Wales,2,School,Y,...,(401) 598-1833,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"8 Abbott Park Place,Providence,RI","(41.825152599999996, -71.42491442533371, 0.0)"
5,704,NaN,21502,NaN,Naval War College,Naval War College,Naval War,2,School,Y,...,(401) 841-1297,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"686 Cushing Road,Newport,RI","(41.5076757, -71.3291036, 0.0)"
6,1376,NaN,35506,NaN,New England Institute of Technology ...,NE Institute of Technology,NEIT,2,School,Y,...,(401) 738-8990,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 New England Tech Boulevard,East Greenwich,RI","(41.6616, -71.50270757908407, 0.0)"
7,1103,NaN,28502,NaN,Providence College,Providence College,Providence Col.,2,School,Y,...,(401) 865-2105,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"549 River Avenue,Providence,RI","(41.843295833333336, -71.44055416666667, 0.0)"
8,1104,NaN,28503,NaN,Rhode Island School of Design,Rhode Island School of Design,RISD,2,School,Y,...,(401) 454-6406,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"2 College Street,Providence,RI","(41.826021, -71.407026, 0.0)"
9,1602,NaN,96501,NaN,Roger Williams University,Roger Williams University,Roger Williams,2,School,Y,...,(401) 254-3158,NaN,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Old Ferry Road,Bristol,RI","(41.642946, -71.259916, 0.0)"


In [19]:
df[['latitude', 'longitude']] = pd.DataFrame(df['point'].apply(
    lambda x: (x[0], x[1]) if x is not None else (None, None)
).tolist(), index=df.index)

In [20]:
df

,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,group_ID,listing_order,sch_sub_type_ID,sch_sub_type_name,role_sort_order,source,address,point,latitude,longitude
0,3585,42.0,70506,NaN,Brigham Young University,Brigham Young University,Brigham Young U,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"Brigham Young University,Provo,UT","(40.25501105, -111.64968264312222, 0.0)",40.255011,-111.649683
1,1102,NaN,28501,NaN,Brown University ...,Brown University,Brown Univ.,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Prospect Street ,Providence,RI","(41.8261477, -71.40384783120373, 0.0)",41.826148,-71.403848
2,1174,NaN,31501,NaN,Bryant University,Bryant University,Bryant Univ,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1150 Douglas Pike,Smithfield,RI","(41.9184668, -71.5365125, 0.0)",41.918467,-71.536513
3,3452,42.0,70505,NaN,Clarkson University,Clarkson University,Clarkson Univ,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,NaN,RIDE_May 3 2023 3:37PM,"8 Clarkson Ave,Potsdam,NY","(44.663631761589166, -74.99410567557055, 0.0)",44.663632,-74.994106
4,1105,NaN,28504,NaN,Johnson and Wales University,Johnson and Wales University,Johnson Wales,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"8 Abbott Park Place,Providence,RI","(41.825152599999996, -71.42491442533371, 0.0)",41.825153,-71.424914
5,704,NaN,21502,NaN,Naval War College,Naval War College,Naval War,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"686 Cushing Road,Newport,RI","(41.5076757, -71.3291036, 0.0)",41.507676,-71.329104
6,1376,NaN,35506,NaN,New England Institute of Technology ...,NE Institute of Technology,NEIT,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 New England Tech Boulevard,East Greenwich,RI","(41.6616, -71.50270757908407, 0.0)",41.661600,-71.502708
7,1103,NaN,28502,NaN,Providence College,Providence College,Providence Col.,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"549 River Avenue,Providence,RI","(41.843295833333336, -71.44055416666667, 0.0)",41.843296,-71.440554
8,1104,NaN,28503,NaN,Rhode Island School of Design,Rhode Island School of Design,RISD,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"2 College Street,Providence,RI","(41.826021, -71.407026, 0.0)",41.826021,-71.407026
9,1602,NaN,96501,NaN,Roger Williams University,Roger Williams University,Roger Williams,2,School,Y,...,NaN,NaN,10,Private Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,"1 Old Ferry Road,Bristol,RI","(41.642946, -71.259916, 0.0)",41.642946,-71.259916


In [21]:
df.to_csv('RI_colleges_loc.csv')

In [18]:
df = df[df['point'].isnull()]
geolocator2 = GeoNames(username='BrownUniversityLibrary')
df['location'] = df['address_location1'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Community College of R.I.',), **{}).
Traceback (most recent call last):
  File "/Users/eriq/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/eriq/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "/Users/eriq/opt/anaconda3/lib/python3.9/http/client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/eriq/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
 

,org_ID,parent_ID,code,finance_code,name,name_short_30,name_short_15,org_type_ID,org_type,active,...,contact_phone3,contact_fax,contact_email,group_ID,listing_order,sch_sub_type_ID,sch_sub_type_name,role_sort_order,source,point
13,590,2950.0,17501,NaN,Community College of R.I. - Flanagan Campus,Community College of R.I.,CCRI-Lincoln,2,School,Y,...,NaN,(401) 333-7111,NaN,NaN,NaN,24,State Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,None
14,1375,2950.0,35501,NaN,Community College of R.I. - Knight Campus,Community College of R.I.,CCRI,2,School,Y,...,NaN,(401) 825-2166,NaN,NaN,NaN,24,State Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,None
15,1108,2950.0,28507,NaN,Community College of R.I. - Liston Campus,Community College of RI - List,CCRI-Providence,2,School,Y,...,NaN,(401) 455-6190,NaN,NaN,NaN,24,State Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,None
16,705,2950.0,21503,NaN,Community College of R.I.-Newport County Campus,Community College of RI-New,CCRI-Newport,2,School,Y,...,NaN,(401) 851-1662,NaN,NaN,NaN,24,State Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,None
22,2385,NaN,28508,NaN,URI/Alan Shawn Feinstein College of Continuing...,URI/Alan Shawn Feinstein,URI/Alan Shawn,2,School,Y,...,NaN,(401) 277-5099,NaN,NaN,NaN,24,State Colleges and Universities,5.0,RIDE_May 3 2023 3:37PM,None


In [3]:
df['location_address1']

0             Brigham Young University
1     1 Prospect Street               
2                    1150 Douglas Pike
3                       8 Clarkson Ave
4                  8 Abbott Park Place
5                     686 Cushing Road
6         1 New England Tech Boulevard
7                     549 River Avenue
8                     2 College Street
9                     1 Old Ferry Road
10              100 Ochre Point Avenue
11                      84 Alford Road
12                  1250 Turner Street
13              1762 Louisquisset Pike
14                     400 East Avenue
15                     1 Hilton Street
16              1 John H. Chafee Blvd.
17                          99 Main St
18      600 Mt. Pleasant Avenue       
19                1111 Franklin Street
20                           Iowa City
21                    35 Campus Avenue
22                80 Washington Street
Name: location_address1, dtype: object